# Working with ARCH Derivatives

In this notebook we'll do some analyses of the files we got from the COVID19 IIPC collection. It is based on the [original code from Nick Ruest and the Archive Unleashed team](https://github.com/archivesunleashed/notebooks).


# Datasets

In this section, we don't download the data, as it is too big. The data we are using should have been doanload prior and stored, in *.gz* files in a directory called 'data' which is in the same folder as this notebook.

Unzip the data. We unzip them here, as there seem to be a bug in macOS when you use the Apple provided tools.

In [ ]:
!gunzip data/*

Let's check our `data` directory, and make sure they've downloaded.


In [ ]:
!ls -1 data

# Environment

Next, we'll setup our environment so we can load our derivatives into [pandas](https://pandas.pydata.org).

In [ ]:
import pandas as pd

# Data Table Display

> We are working out of Google Colab. So this part is usable only if you use it within Colab. But with files that are that large, it seems illusory to work on Colab. I let here this code 'in case of'.

Colab includes an extension that renders pandas dataframes into interactive displays that can be filtered, sorted, and explored dynamically. This can be very useful for taking a look at what each DataFrame provides!

Data table display for pandas dataframes can be enabled by running:
```python
%load_ext google.colab.data_table
```
and disabled by running
```python
%unload_ext google.colab.data_table
```

In [ ]:
%load_ext google.colab.data_table

# Loading our ARCH Datasets as DataFrames

---



Next, we'll setup our datasets as pandas DataFrames to work with, and show a preview of each using the Data Table Display.

Each block of derivative commands create a variable. That variable is a DataFrame with all of the information from a given derivative. After the DataFrame is created, a preview of it is shown.

## Collection

A basic overview of the collection.


### Domain Frequency

Provides the following columns:

* domain
* count

In [ ]:

domain_frequency = pd.read_csv("data/domain-frequency.csv")
domain_frequency

## Network
These derivative files provide network graph data for analysis, and offer an opportunity to explore the way websites link to each other.



### Domain Graph

Provides the following columns:

* crawl date
* source domain
* target domain
* count

In [ ]:
domain_graph = pd.read_csv("data/domain-graph.csv")
domain_graph

### Image Graph

Provides the following columns:

* crawl date
* source of the image (where it was hosted)
* the URL of the image
* the alternative text of the image

**Due to the size of the graph, this will take time.**

In [ ]:
image_graph = pd.read_csv("data/image-graph.csv")
image_graph

### Web Graph

Provides the following columns:

* crawl date
* source
* target
* anchor text

Note that this contains all links and is not aggregated into domains.

**Due to the size of the graph, this will take time.**

In [ ]:
web_graph = pd.read_csv("data/web-graph.csv")
web_graph

## File Formats

These derivatives contain information on certain types of binary files found within a web archive.

### Audio

Provides the following columns:

* crawl date
* URL of the audio file
* filename
* audio extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* audio MD5 hash
* audio SHA1 hash





In [ ]:
audio = pd.read_csv("data/audio-information.csv")
audio

### Images

Provides the following columns:

* crawl date
* URL of the image
* filename
* image extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* image width
* image height
* image MD5 hash
* image SHA1 hash

In [ ]:
images = pd.read_csv("data/image-information.csv")
images

### PDFs

Provides the following columns:

* crawl date
* URL of the PDF file
* filename
* PDF extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* PDF MD5 hash
* PDF SHA1 hash

In [ ]:
pdf = pd.read_csv("data/pdf-information.csv")
pdf

### PowerPoint (all software types) information

Provides the following columns:

* crawl date
* URL of a PowerPoint or similar file
* filename
* PowerPoint or similar file extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* PowerPoint or similar file MD5 hash
* PowerPoint or similar file SHA1 hash

In [ ]:
powerpoint = pd.read_csv("data/powerpoint-information.csv")
powerpoint

### Spreadsheets

Provides the following columns:

* crawl date
* URL of the spreadsheet file
* filename
* spreadsheet extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* spreadsheet MD5 hash
* spreadsheet SHA1 hash


In [ ]:
spreadsheet = pd.read_csv("data/spreadsheet-information.csv")
spreadsheet

### Videos

Provides the following columns:

* crawl date
* URL of the video file
* filename
* video extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* video MD5 hash
* video SHA1 hash


In [ ]:
video = pd.read_csv("data/video-information.csv")
video

### Word Documents (all software types)

Provides the following columns:

* crawl date
* URL of the word document or similar file
* filename
* word document or similar file extension
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* word document or similar file MD5 hash
* word document or similar file SHA1 hash


In [ ]:
word = pd.read_csv("data/word-document-information.csv")
word

## Text

This derivative provides access to the "plain text" of a collection, extracted from web page HTML.

### Web Pages

Provides the following columns:

* crawl date
* web domain
* URL
* MIME type as provided by the web server
* MIME type as detected by Apache TIKA
* content (HTTP headers and HTML removed)

**This file is HUGE! Be patient!**

In [ ]:
web_pages = pd.read_csv("data/web-pages.csv")
web_pages

# Data Analysis

Now that we have all of our datasets loaded up, we can begin to work with them!

## Counting total files, and unique files

Let's take a quick look at how to count items in DataFrames, and use total and unique files as an example to work with.

It's definitely work checking out the [pandas documentation](https://pandas.pydata.org/docs/index.html). There are a lot of good examples available, along with a robust [API reference](https://pandas.pydata.org/docs/reference/index.html#api).


#### How many images are in this collection?

We can take our `images` variable try a couple of functions to get the same answer.

1.   `len(images.index)`
  * Get the length of the DataFrame's index.
2.   `images.shape[0]`
  * Get the shape or dimensionality of the DataFrame, and take the first item in the tuple.
3.  `images.count()`
  * Count the number of rows for each column.



In [ ]:
len(images.index)

In [ ]:
images.shape[0]

In [ ]:
images.count()

 #### How many unique images are in the collection?

 We can see if an image is unique or not by computing an [MD5 hash](https://en.wikipedia.org/wiki/MD5#MD5_hashes) of it, and comparing them. The exact same image might have a filename of `example.jpg` or `foo.jpg`. If the hash is computed for each, we can see that even with different file names, they are actually the same image. So, since we have both a `MD5` and `SHA1` hash column available in our DataFrame, we can just find the unique values, and count them!




In [ ]:
len(images.md5.unique())

#### What are the top 10 most occurring images in the collection?

Here we can take advantage of [`value_counts()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html) to provide us with a list of MD5 hashes, and their respective counts.

In [ ]:
images["md5"].value_counts().head(10)


#### What's the information around all of the occurances of `d89746888da2d9510b64a9f031eaecd5`?

What, you mean you don't know what `d89746888da2d9510b64a9f031eaecd5` means? 

Let's find those images in the DataFrame. We can here see some of the filenames used, it's dimensions, and it's URL.


In [ ]:
images.loc[images["md5"] == "d89746888da2d9510b64a9f031eaecd5"]

### What does `377d257f2d2e294916143c069141c1c5` look like?

Let's grab the live web URL for the image, and then see if we can display it in a markdown cell.


In [ ]:
pd.options.display.max_colwidth = None
one_image = images.loc[images["md5"] == "377d257f2d2e294916143c069141c1c5"].head(1)
one_image["url"]

![377d257f2d2e294916143c069141c1c5](https://analytics.twitter.com/i/adsct?txn_id=l4o6d&p_id=Twitter)

Well, yes, this is a transparent GIF.


Another point of examination with the `images` DataFrame is the `height` and `width` columns. You could take a look at the largest images, or even `0x0` images, and potentially `spacer.gif` occurrences!

* “[The invention and dissemination of the spacer gif: implications for the future of access and use of web archives](https://link.springer.com/article/10.1007/s42803-019-00006-8)”
* "[GeoCities and the spacer.gif](https://ruebot.net/post/geocities-and-the-spacer-gif/)"


#### What are the top 10 most occuring filenames in the collection?

Note that this is of course different than the MD5 results up above. Here we are focusing _just_ on filename. So `cover.jpg` for example, might actually be referring to different images who happen to have the same name.

Here we can use `value_counts()` again, but this time we'll create a variable for the top filenames so we can use it later.



In [ ]:
top_filenames = images["filename"].value_counts().head(10)
top_filenames

#### Let's create our first graph!

We'll plot the data first with pandas [plot](https://pandas.pydata.org/docs/reference/api/pandas.Series.plot.html) functionality, and then plot the data with [Altair](https://altair-viz.github.io/).

In [ ]:
top_filenames_chart = top_filenames.plot.bar(figsize=(25, 10))

top_filenames_chart.set_title("Top Filenames", fontsize=22)
top_filenames_chart.set_xlabel("Filename", fontsize=20)
top_filenames_chart.set_ylabel("Count", fontsize=20)

Now let's setup Altair, and plot the data with Altair. Altair is useful for creating vizualization since they can be easily exported as a PNG or SVG.

In [ ]:
import altair as alt

In [ ]:
top_filenames_altair = (
    images["filename"]
    .value_counts()
    .head(10)
    .rename_axis("Filename")
    .reset_index(name="Count")
)

filenames_bar = (
    alt.Chart(top_filenames_altair)
    .mark_bar()
    .encode(x=alt.X("Filename:O", sort="-y"), y=alt.Y("Count:Q"))
)

filenames_rule = (
    alt.Chart(top_filenames_altair).mark_rule(color="red").encode(y="mean(Count):Q")
)


filenames_text = filenames_bar.mark_text(align="center", baseline="bottom").encode(
    text="Count:Q"
)

(filenames_bar + filenames_rule + filenames_text).properties(
    width=1400, height=700, title="Top Filenames"
)

#### How about a file format distribution?

What _kind_ of image files are present? We can discover this by checking their "media type", or [MIME type](https://en.wikipedia.org/wiki/Media_type). 






In [ ]:
image_mime_types = (
    images["mime_type_tika"]
    .value_counts()
    .head(5)
    .rename_axis("MIME Type")
    .reset_index(name="Count")
)

image_mimes_bar = (
    alt.Chart(image_mime_types)
    .mark_bar()
    .encode(x=alt.X("MIME Type:O", sort="-y"), y=alt.Y("Count:Q"))
)

image_mime_rule = (
    alt.Chart(image_mime_types).mark_rule(color="red").encode(y="mean(Count):Q")
)

image_mime_text = image_mimes_bar.mark_text(align="center", baseline="bottom").encode(
    text="Count:Q"
)

(image_mimes_bar + image_mime_rule + image_mime_text).properties(
    width=1400, height=700, title="Image File Format Distribution"
)

## Let's take a look at the domain frequency derivative.

#### What does the distribution of domains look like?

Here we can see which domains are the most frequent within the collection.

In [ ]:
top_domains = domain_frequency.sort_values("count", ascending=False).head(10)

top_domains_bar = (
    alt.Chart(top_domains)
    .mark_bar()
    .encode(
        x=alt.X("domain:O", title="Domain", sort="-y"),
        y=alt.Y("count:Q", title="Count, Mean of Count"),
    )
)

top_domains_rule = (
    alt.Chart(top_domains).mark_rule(color="red").encode(y="mean(count):Q")
)

top_domains_text = top_domains_bar.mark_text(align="center", baseline="bottom").encode(
    text="count:Q"
)

(top_domains_bar + top_domains_rule + top_domains_text).properties(
    width=1400, height=700, title="Domains Distribution"
)

### Top Level Domain Analysis

pandas allows you to create new columns in a DataFrame based off of existing data. This comes in handy for a number of use cases with the available data that we have. In this case, let's create a new column, `tld`, which is based off an existing column, 'domain'. This example should provide you with an implementation pattern for expanding on these datasets to do further research and analysis.

A [top-level domain](https://en.wikipedia.org/wiki/Top-level_domain) refers to the highest domain in an address - i.e. `.ca`, `.com`, `.org`, or yes, even `.pizza`.

Things get a bit complicated, however, in some national TLDs. While `qc.ca` (the domain for Quebec) isn't really a top-level domain, it has many of the features of one as people can directly register under it. Below, we'll use the command `suffix` to include this. 

> You can learn more about suffixes at https://publicsuffix.org.

We'll take the `domain` column and extract the `tld` from it with [`tldextract`](https://github.com/john-kurkowski/tldextract).

First we'll add the [`tldextract`](https://github.com/john-kurkowski/tldextract) library to the notebook. Then, we'll create the new column.

In [ ]:
%%capture

!pip install tldextract

In [ ]:
import tldextract

domain_frequency["tld"] = domain_frequency.apply(
    lambda row: tldextract.extract(row.domain).suffix, axis=1
)
domain_frequency

#### Next, let's count the distict TLDs.


In [ ]:
tld_count = domain_frequency["tld"].value_counts()
tld_count

#### Next, we'll plot the TLD count.


In [ ]:
tld_count = (
    domain_frequency["tld"]
    .value_counts()
    .rename_axis("TLD")
    .reset_index(name="Count")
    .head(10)
)

tld_bar = (
    alt.Chart(tld_count)
    .mark_bar()
    .encode(x=alt.X("TLD:O", sort="-y"), y=alt.Y("Count:Q"))
)

tld_rule = alt.Chart(tld_count).mark_rule(color="red").encode(y="mean(Count):Q")

tld_text = tld_bar.mark_text(align="center", baseline="bottom").encode(text="Count:Q")

(tld_bar + tld_rule + tld_text).properties(
    width=1400, height=700, title="Top Level Domain Distribution"
)

## Web Crawl Frequency

Let's see what the crawl frequency looks like by examining the `web_pages` DataFrame. First we'll create a new DataFrame by extracting the `crawl_date` and `domain` columns, and count the occurances of each domain and date combination.

In [ ]:
crawl_sites = web_pages[["crawl_date", "domain"]]
crawl_sites = crawl_sites.value_counts().reset_index()
crawl_sites.columns = ["Date", "Site", "Count"]
crawl_sites

Next, we'll create a stacked bar chart where each bar will show the distribution of pages in that crawl by top-level domain.

**NOTE**: Charts like this one work a lot better with collections that have more than 1 or 2 crawl dates. The temporal aspect is definitely something to take into consideration with each of the examples provided in this notebook.

In [ ]:
## Altair has a default limit of 5000 rows, and this DataFrame is ~7700 rows, so we're going to disable the max allowed rows.
alt.data_transformers.disable_max_rows()

crawl_chart = (
    alt.Chart(crawl_sites)
    .mark_bar()
    .encode(
        x="Date:O",
        y="Count:Q",
        color="Site",
        tooltip="Site",
        order=alt.Order("Site", sort="descending"),
    )
)

crawl_chart.properties(width=1400, height=700, title="Web Crawl Frequency")

## Examining the Domain Graph

Remember the hyperlink Domain graph? Let's look at the web graph columns again.



In [ ]:
domain_graph


### What are the most frequent `source` and `target` combinations?

In [ ]:
top_links = domain_graph[["source", "target"]].value_counts().head(150).reset_index()
top_links.columns = ["source", "target", "count"]
top_links

## Can we create a network graph visualization with the data we have?

Yes! We can take advantage [NetworkX](https://networkx.org/documentation/stable/index.html) to create some basic graphs.

NetworkX is *really* powerful, so there is a lot more that can be done with it than what we're demonstrating here.

First we'll import `networkx` as well as `matplotlib.pyplot`.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

We can take advantage of [`from_pandas_edgelist`](https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html) here since our three graph derivatives are edge tables, and initialize our graph.


In [ ]:
G = nx.from_pandas_edgelist(
    top_links, source="source", target="target", edge_key="target", edge_attr="count"
)

Setup our graph, and draw it!


In [ ]:
pos = nx.spring_layout(G, k=25)
options = {
    "node_size": 1000,
    "node_color": "#bc5090",
    "node_shape": "o",
    "alpha": 0.5,
    "linewidths": 4,
    "font_size": 10,
    "font_color": "black",
    "width": 2,
    "edge_color": "grey",
}

plt.figure(figsize=(15, 15))

nx.draw(G, pos, with_labels=True, **options)

labels = {e: G.edges[e]["count"] for e in G.edges}
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

plt.show()

## Text Analysis

Next, we'll do some basic text analysis with our `web_pages` DataFrame with `nltk` and`spaCy`, and end with a word cloud.


In [ ]:
import re

import nltk

In [ ]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

We'll drop the `NaN` values in our DataFrame to clean things up a bit.

In [ ]:
web_pages = web_pages.dropna()
web_pages

We need to set the [`mode.chained_assignment`](https://pandas.pydata.org/docs/user_guide/options.html?highlight=chained_assignment) to `None` now to silence some exception errors that will come up.

In [ ]:
pd.options.mode.chained_assignment = None

Next, we'll setup a tokenizer which will split on words, and create a new column which is the tokenized text.

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [ ]:
web_pages["content_tokenized"] = web_pages["content"].map(tokenizer.tokenize)

Now well create a column with the tokenized value count.

In [ ]:
web_pages["content_tokens"] = web_pages["content_tokenized"].apply(lambda x: len(x))

### Basic word count statistics with pandas!

Now we can use the power of pandas [Statisitcal functions](https://pandas.pydata.org/docs/user_guide/computation.html) to show us some basic statistics about the tokens.

**Mean**

In [ ]:
web_pages["content_tokens"].mean()

**Standard deviation**


In [ ]:
web_pages["content_tokens"].std()

**Max**

In [ ]:
web_pages["content_tokens"].max()

**Min**

In [ ]:
web_pages["content_tokens"].min()

### Pages with most words

Let's create a bar chart that shows the pages with the most words. Here we can see the power of pandas at work, in terms of both analysis and visualization.

First, let's show the query to get the data for our chart.

In [ ]:
word_count = (
    web_pages[["url", "content_tokens"]]
    .sort_values(by="content_tokens", ascending=False)
    .head(25)
)

In [ ]:
word_count

Next, let's create a bar chart of this.

In [ ]:
word_count_bar = (
    alt.Chart(word_count)
    .mark_bar()
    .encode(x=alt.X("url:O", sort="-y"), y=alt.Y("content_tokens:Q"))
)

word_count_rule = (
    alt.Chart(word_count).mark_rule(color="red").encode(y="mean(content_tokens):Q")
)

word_count_text = word_count_bar.mark_text(align="center", baseline="bottom").encode(
    text="content_tokens:Q"
)

(word_count_bar + word_count_rule + word_count_text).properties(
    width=1400, height=700, title="Pages with the most words"
)

### How about NER on the page with the most tokens?

[Named-Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition), or NER, is an exciting field of natural language processing that lets us extract "entities" out of text; the names of people, locations, or organizations.

To do this, we first need to find the pages that have the most tokens.

In [ ]:
word_count_max = (
    web_pages[["url", "content_tokens", "content"]]
    .sort_values(by="content_tokens", ascending=False)
    .head(1)
)
word_count_max["url"]

We'll remove the column width limit so we can check out our content for the page.

In [ ]:
pd.set_option("display.max_colwidth", None)

Let's take a look at our page's content.

In [ ]:
page = word_count_max["content"].astype("unicode").to_string()
page


#### Setup spaCy

We now need to set up [spaCy](https://en.wikipedia.org/wiki/SpaCy), a natural-language processing toolkit.


In [ ]:
import en_core_web_sm
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

nlp.max_length = 1100000

Next we'll run the natual language processor from SpaCy, and then display the NER output. Watch how it finds organizations, people, and beyond!

In [ ]:
ner = nlp(page)
displacy.render(ner, style="ent", jupyter=True)

### Sentiment Analysis

We'll be using the [vaderSentiment](https://github.com/cjhutto/vaderSentiment) library, and [adapting examples](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/04-Sentiment-Analysis.html#) from Melanie Walsh's ["Introduction to Cultural Analytics & Python"](https://melaniewalsh.github.io/Intro-Cultural-Analytics).

In [ ]:
%%capture

!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER
sentimentAnalyser = SentimentIntensityAnalyzer()

We'll create a function, that we'll then apply to a DataFrame to create sentiment analysis scores for the `content` column.

In [ ]:
def calculate_sentiment(text):
    # Run VADER on the text
    scores = sentimentAnalyser.polarity_scores(text)
    # Extract the compound score
    compound_score = scores['compound']
    # Return compound score
    return compound_score

Since it will take some time to run the sentiment analysis on the entire `web_pages` DataFrame, we'll create a sample from `web_pages`, and run the sentiment analysis on the for demostration purposes.

In [ ]:
web_pages_sample = web_pages.sample(500)

In [ ]:
web_pages_sample['sentiment_score'] = web_pages_sample['content'].apply(calculate_sentiment)

Let's see what the the scores look like.

In [ ]:
web_pages_sample[['sentiment_score', 'content']]

Finally, let's plot the sentiment score.

In [ ]:
sentiment_scores = web_pages_sample[['sentiment_score']].value_counts().head(10).reset_index()
sentiment_scores = sentiment_scores.rename({"sentiment_score": "Sentiment Score", 0: "Count"}, axis=1)

sentiment_chart = (
    alt.Chart(sentiment_scores)
    .mark_circle()
    .encode(
        x=alt.X("Sentiment Score:Q", bin=True),
        y=alt.Y("Count:Q", bin=True),
        size='Count')
)

sentiment_chart.properties(
    width=1400, height=700, title="Sentiment Score Distribution"
)

### Wordcloud

What better way to wrap-up this notebook than create a word cloud!

Word clouds are always fun, right?! They're an interesting way to visualize word frequency, as the more times that a word occurs, the larger it will appear in the word cloud.

Let's setup some dependencies here. We will install the [word_cloud](https://github.com/amueller/word_cloud) library, and setup some stop words via `nltk`.

In [ ]:
%%capture

!pip install wordcloud
from wordcloud import WordCloud, ImageColorGenerator

Let's remove the remove the stopwords from our data.

In [ ]:
stopwords = stopwords.words("english")

In [ ]:
web_pages["stopwords"] = web_pages["content_tokenized"].apply(
    lambda x: [item.lower() for item in x if item not in stopwords]
)

Next we'll pull 500 rows of values from our new column.

In [ ]:
words = web_pages["stopwords"].head(500)

Now we can create a word cloud!

In [ ]:
wordcloud = WordCloud(
    width=2000,
    height=1500,
    scale=10,
    max_font_size=250,
    max_words=100,
    background_color="white",
).generate(str(words))
plt.figure(figsize=[35, 10])
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()